In [ ]:
#Reference: https://medium.com/crypto-currently/lets-make-the-tiniest-blockchain-bigger-ac360a328f4d

In [40]:
import hashlib
import datetime
from flask import Flask
from flask import request
import json

In [126]:
class Block:
    def __init__(self,index,timestamp,data,previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block()
        
    def hash_block(self):
        sha = hashlib.sha256()
        sha.update(str(self.index).encode('utf-8') +
                   str(self.timestamp).encode('utf-8') +
                   str(self.data).encode('utf-8') +
                   str(self.previous_hash).encode('utf-8'))
        return sha.hexdigest()
        

In [91]:
def create_genesis_block():
    return Block(0,datetime.datetime.now(),{
        "proof-of-work":9,
        "transaction":"Genesis Block"},"0")



In [139]:
blockchain=[]
this_nodes_transactions = []

blockchain.append(create_genesis_block())


In [92]:
def next_block(last_block):
    index = last_block.index + 1
    timestamp = datetime.datetime.now()
    data = "New block with index" + str(index)
    previous_hash = last_block.previous_hash
    return Block(index,timestamp,data,previous_hash)

In [102]:
blockchain[0].data

{'proof-of-work': 9, 'transaction': 'Genesis Block'}

In [50]:
genesis_block = blockchain[0]

#adding 20 blocks
for i in range(1,20):
    blockchain.append(next_block(blockchain[i-1]))
    print(f'Block {i} has been added to the chain')
    print(f'Hash value: {blockchain[i].hash}')
    #c0e1dccff39ef193ce827f7f9ea75940af3ab2a43f1167cae9924f4cb37d7cb1

Block 1 has been added to the chain
Hash value: 9daa706279b7338be8bb2df9a6c45800cd338723cf327c922e0c7f6232ccb57f
Block 2 has been added to the chain
Hash value: 821dac66b3497ff5f083657874d865638a152e38c4b76ee57fe4c46344f8c109
Block 3 has been added to the chain
Hash value: 99852bcd6e02593674a6e98bed64edf8512e4b57cc447c7e3c974b4f746a1527
Block 4 has been added to the chain
Hash value: da9fb49c4463eddefe49ec883d9508b2eea00a50334994bb6f030cc87ae5625f
Block 5 has been added to the chain
Hash value: 9e0bf1f3b3473962e84f4d1b37b377f7298b0d981175f63cab7e748e8401bdfe
Block 6 has been added to the chain
Hash value: 18d65c37bce524fbed21b509ed4f0301f9876d67fe9c240e21860fb0fc789e74
Block 7 has been added to the chain
Hash value: d9239ab8acb3e8c0ee51bb2cb1aaf793eb7be8c5d361eed03da35ddac60664e8
Block 8 has been added to the chain
Hash value: 4796f7888137c6fa44166fdf8212414cf1deffd66daefc78163e1874a1003592
Block 9 has been added to the chain
Hash value: 0e1cd874f365ff10e2134138f4dd8f17118b521b9591c148

In [147]:
node = Flask(__name__)

@node.route('/transaction',methods=['POST'])
def submit_transaction():
    if request.method == 'POST':
        #extract data
        print(request.get_json())
        new_transaction = request.get_json()
        print(new_transaction)
        this_nodes_transactions.append(new_transaction)
        print('New Transaction')
        print(f"From: {new_transaction['from']}")
        print(f"To: {new_transaction['to']}")
        print(f"Amount: {new_transaction['amount']}")
        return 'Transaction Submitted Succesfully!'
              
miner_address = 'abcs-random-miner-1234'

def proof_of_work(last_proof):
    incrementor = last_proof + 1     
    while not((incrementor % 9 == 0) & (incrementor % last_proof == 0)):
        incrementor += 1
    return incrementor

@node.route('/mine',methods=['GET'])
def mine():
    last_block = blockchain[len(blockchain) - 1]
    last_proof = last_block.data['proof-of-work']
    proof = proof_of_work(last_proof)
    this_nodes_transactions.append(
    {
        "from":"network",
        "to":miner_address,
        "amount":1
    })
    new_block_data = {
        "proof-of-work":proof,
        "transactions":list(this_nodes_transactions)
        
    }
    this_nodes_transactions[:]=[]
   
    mined_block = Block(
                        last_block.index + 1,
                        datetime.datetime.now(),
                        new_block_data,
                        last_block.hash
                        )
    blockchain.append(mined_block)
    

    return json.dumps({
        "index":str(mined_block.index),
        "timestamp":str(mined_block.timestamp),
        "data":str(new_block_data),
        "hash":str(last_block.hash)
    })+ "\n"
        
@node.route('/blocks',methods=['GET'])
def get_blocks():
    blocks = []
    for i in range(len(blockchain)):
              block = blockchain[i]
              block = {
                  "index":block.index,
                  "data":block.data
              }
              blocks.append(block)
    print(blocks)
    return json.dumps(blocks)

def find_new_chains():
    other_chains = []
    for node_url in peer_nodes:
        block = requests.get(node_url + "/blocks").content
        other_chains.append(block)
    return other_chains

def consensus():
    other_chains = find_new_chain()
    longest_chain = blockchain
    for chain in other_chains:
        if(len(longest_chain) < chain):
            longest_chain = chain
    blockchain = longest_chain
              
        
        
node.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jun/2020 01:39:13] "GET /blocks HTTP/1.1" 200 -


[{'index': 0, 'data': {'proof-of-work': 9, 'transaction': 'Genesis Block'}}, {'index': 1, 'data': {'proof-of-work': 18, 'transactions': [{'from': 'network', 'to': 'abcs-random-miner-1234', 'amount': 1}]}}, {'index': 2, 'data': {'proof-of-work': 36, 'transactions': [{'from': 'sdf', 'to': 'geets', 'amount': 20}, {'from': 'network', 'to': 'abcs-random-miner-1234', 'amount': 1}]}}]


127.0.0.1 - - [23/Jun/2020 01:39:34] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 01:39:37] "GET /blocks HTTP/1.1" 200 -


[{'index': 0, 'data': {'proof-of-work': 9, 'transaction': 'Genesis Block'}}, {'index': 1, 'data': {'proof-of-work': 18, 'transactions': [{'from': 'network', 'to': 'abcs-random-miner-1234', 'amount': 1}]}}, {'index': 2, 'data': {'proof-of-work': 36, 'transactions': [{'from': 'sdf', 'to': 'geets', 'amount': 20}, {'from': 'network', 'to': 'abcs-random-miner-1234', 'amount': 1}]}}, {'index': 3, 'data': {'proof-of-work': 72, 'transactions': [{'from': 'network', 'to': 'abcs-random-miner-1234', 'amount': 1}]}}]


In [133]:
import json

json.dumps()

In [134]:
json.dumps()

TypeError: dumps() missing 1 required positional argument: 'obj'